In [1]:
Machine Learning in Spark!

Linear Regression

Data -> This model is expected to predict crew number required on the basis of certain given features.

In [2]:
#Staring Spark Session and import necessary libraries

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer # For String value/Categorical Value
from pyspark.ml.linalg import Vectors #for feature dense vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression #For Linear Model

spark = SparkSession.builder.appName('lr_cruise').getOrCreate()




In [3]:
#Importing the data file and refering the schema in the data and header as row 1

data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv',inferSchema=True,header=True)

#Checking data types
data.printSchema()

In [4]:
#Indexing categorical column(Cruise Line)
indexer = StringIndexer(inputCol = 'Cruise_line',outputCol = 'categoryIndex')
indexed = indexer.fit(data).transform(data)
indexed.head(2)

In [5]:
#Taking columns for creating input dense vector

indexed.columns #only numerical values


In [6]:
#Taking all the features except name, cruise_line and crew(output vector)
assembler = VectorAssembler(inputCols =['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'categoryIndex'],outputCol='features')

In [7]:
#Tranforming all the input features into dense vector
output = assembler.transform(indexed)
#again checking the schema
output.printSchema()

In [8]:
#Machine Learning Lib of Spark only takes one input vector(dense vector)

final_data = output.select(['features','crew'])

#Splitting the data randomly

train_data , test_data = final_data.randomSplit([0.7,0.3])

train_data.describe().show()
test_data.describe().show()

In [9]:
#here target is crew member, fitting and evalutaing the model

lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train_data)
test_result = lr_model.evaluate(test_data)

In [10]:
#printing adjusted r2
#This is showing highly efficient model
test_result.r2


In [11]:
#predicting against unlabelled data

unlabeled_data = test_data.select('features')
prediction = lr_model.transform(unlabeled_data)
prediction.show()
test_data.select('crew').show()